In [1]:
import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,  HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from tqdm import tqdm
from sklearn.metrics import roc_auc_score

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
data = pd.read_csv("./data/competition_data.csv")

data.drop(columns=['benefit', 'category_id', 'deal_print_id','etl_version', 'full_name','product_id'
                   , 'item_id', 'main_picture', 'site_id', 'uid', 'user_id', 'title', 'tags', 'warranty'],
                     axis= "columns", inplace=True)

data['conversion'] = data['conversion'].astype('bool')
data["print_server_timestamp"] = pd.to_datetime(data["print_server_timestamp"])
data["hour"] = data["print_server_timestamp"].dt.hour
data["day"] = data["print_server_timestamp"].dt.day
#data["minute"] = data["print_server_timestamp"].dt.minute
data["month"] = data["print_server_timestamp"].dt.month
data.drop(columns=["print_server_timestamp", 'date', "domain_id"], axis= "columns", inplace=True)
data = pd.get_dummies(data,columns = ["listing_type_id", "logistic_type", "platform"],dummy_na = False, dtype = int)
data = pd.get_dummies(data,columns = ["is_pdp"],dummy_na = True, dtype = bool )

In [3]:
train_data = data[data["ROW_ID"].isna()]
test_data = data[data["ROW_ID"].notna()]

x_train = train_data.drop(columns=["conversion", "ROW_ID"])
y_train = train_data["conversion"]
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=3456)
X_test = test_data.drop(columns=["conversion"])
y_test = test_data["conversion"]

In [8]:
clf_sgd = make_pipeline(StandardScaler(),
                    SimpleImputer(strategy='mean'),
                    SGDClassifier(max_iter=1000, tol=1e-3, loss="log_loss", random_state=42))
clf_sgd.fit(X_train, Y_train)
print(clf_sgd.score(X_val, Y_val))
print(roc_auc_score(Y_val, clf_sgd.predict_proba(X_val)[:, clf_sgd.classes_== 1]))

0.9049871379968467
0.8631369716405869


In [10]:
knn = make_pipeline(StandardScaler(),
                    SimpleImputer(strategy='mean'),
                    KNeighborsClassifier(n_neighbors=50, weights="uniform"))
knn.fit(X_train, Y_train)
print(knn.score(X_val, Y_val))
print(roc_auc_score(Y_val, knn.predict_proba(X_val)[:, knn.classes_== 1]))

0.9095510745996183
0.8525217035020392


In [11]:
log_reg = make_pipeline(StandardScaler(),
                    SimpleImputer(strategy='mean'),
                    LogisticRegression(max_iter = 1000))
log_reg.fit(X_train, Y_train)
print(log_reg.score(X_val, Y_val))
print(roc_auc_score(Y_val, log_reg.predict_proba(X_val)[:, log_reg.classes_== 1]))

0.9071169750781402
0.8651776709620094


In [12]:
xgb_clas = make_pipeline(StandardScaler(),
                    #SimpleImputer(strategy='mean'),
                    xgb.XGBClassifier(n_estimators=100))
xgb_clas.fit(X_train, Y_train)
print(xgb_clas.score(X_val, Y_val))
print(roc_auc_score(Y_val, xgb_clas.predict_proba(X_val)[:, xgb_clas.classes_== 1]))

/Users/tomasglauberman/opt/anaconda3/envs/td6/lib/python3.11/site-packages/xgboost/data.py:440: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


0.909910657483473
0.8928563518616219


In [10]:
# svm = make_pipeline(StandardScaler(),
#                     SimpleImputer(strategy='mean'),
#                     svm.SVC(kernel='rbf'))
# svm.fit(X_train, Y_train)
# print(svm.score(X_val, Y_val))

In [13]:
grad_boosting = make_pipeline(StandardScaler(),
                    SimpleImputer(strategy='mean'),
                    GradientBoostingClassifier(random_state=0))
grad_boosting.fit(X_train, Y_train)
print(grad_boosting.score(X_val, Y_val))
print(roc_auc_score(Y_val, grad_boosting.predict_proba(X_val)[:, grad_boosting.classes_== 1]))

0.9093021326031034
0.8808584300735637


In [14]:
rand_for = make_pipeline(StandardScaler(),
                    SimpleImputer(strategy='mean'),
                    RandomForestClassifier(n_estimators=200, random_state=0))
rand_for.fit(X_train, Y_train)
print(rand_for.score(X_val, Y_val))
print(roc_auc_score(Y_val, rand_for.predict_proba(X_val)[:, rand_for.classes_== 1]))

0.9018615329294941
0.8713415860489206


In [7]:
j = 0
for i in tqdm(models):
    model = i
    cv = KFold(n_splits=5, random_state=0, shuffle=True)
    KFold_Score[classifiers[j]] = (cross_val_score(model, X_train, Y_train, scoring = 'roc-auc', cv=cv))
    j = j+1

  0%|          | 0/8 [00:00<?, ?it/s]